# Retraining Cellpose on Custom Data

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/05_segmentation/deep_learning/cellpose_retraining_notebook.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/05_segmentation/deep_learning/cellpose_retraining_notebook.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "tifffile",
#     "cellpose"
# ]
# ///

## Overview

In this section...

If you do not have an Apple Silicon Mac or a GPU we suggest to run this notebook in Google Colab.

...

You need pair of raw data and respective labels.
You can generate the labels in any way you want.
you can for example use the Cellpose GUI to manually update/create the labels (https://cellpose.readthedocs.io/en/latest/gui.html#training-your-own-cellpose-model) or for example you can use other software for data annotation (e.g. FIND and ADD EXAMPLES)

The images we will use for this section can be downloaded from the <a href="../../../_static/data/05_segmentation_cellpose_training.zip" download> <i class="fas fa-download"></i> Cellpose Training Dataset</a> (it contains both training and test images).

## Data Handling

Training dataset vs Test dataset

## Train new model

## Evaluate on test data

TO REMOVE: <a href="https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/train_Cellpose-SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>